In [73]:
## IMPORT NECESSARY LIBRARIES
from google.colab import files
import zipfile
from io import BytesIO
import io
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, GlobalAvgPool2D
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix


In [74]:
## UPLOADING THE ZIPFILE
uploaded = files.upload()

Saving cats_dogs.zip to cats_dogs (2).zip


In [76]:
## UNZIPPING THE FILE
data = zipfile.ZipFile(io.BytesIO(uploaded['cats_dogs (2).zip']))
data.extractall()
data.printdir()
data.close()

File Name                                             Modified             Size
cats_dogs/                                     2024-12-26 16:07:48            0
cats_dogs/test_dir/                            2024-12-26 16:07:14            0
cats_dogs/test_dir/test_cats/                  2024-12-26 16:03:12            0
cats_dogs/test_dir/test_cats/871.jpg           2024-12-26 16:03:12        26546
cats_dogs/test_dir/test_cats/872.jpg           2024-12-26 16:03:12        24848
cats_dogs/test_dir/test_cats/873.jpg           2024-12-26 16:03:12        30621
cats_dogs/test_dir/test_cats/874.jpg           2024-12-26 16:03:12         5923
cats_dogs/test_dir/test_cats/875.jpg           2024-12-26 16:03:12        40763
cats_dogs/test_dir/test_cats/876.jpg           2024-12-26 16:03:12        38996
cats_dogs/test_dir/test_cats/877.jpg           2024-12-26 16:03:12        38218
cats_dogs/test_dir/test_cats/878.jpg           2024-12-26 16:03:12        43619
cats_dogs/test_dir/test_cats/879.jpg    

In [77]:
## PATH CREATION
base_dir = 'cats_dogs'
train_dir = os.path.join(base_dir, 'train_dir')
test_dir = os.path.join(base_dir, 'test_dir')
train_cats_dir = os.path.join(train_dir, 'train_cats')
train_dogs_dir = os.path.join(train_dir, 'train_dogs')
test_cats_dir = os.path.join(test_dir, 'test_cats')
test_dogs_dir = os.path.join(test_dir, 'test_dogs')

tr_cats_fnames = os.listdir(train_cats_dir)
tr_dogs_fnames = os.listdir(train_dogs_dir)
ts_cats_fnames = os.listdir(test_cats_dir)
ts_dogs_fnames = os.listdir(test_dogs_dir)


In [78]:
print(tr_cats_fnames)
print(tr_dogs_fnames)
print(ts_cats_fnames)
print(ts_dogs_fnames)

['20.jpg', '3.jpg', '18.jpg', '11.jpg', '17.jpg', '10.jpg', '1.jpg', '7.jpg', '14.jpg', '6.jpg', '13.jpg', '16.jpg', '21.jpg', '5.jpg', '9.jpg', '22.jpg', '25.jpg', '24.jpg', '23.jpg', '19.jpg', '0.jpg', '12.jpg', '2.jpg', '4.jpg', '15.jpg', '8.jpg']
['20.jpg', '3.jpg', '18.jpg', '11.jpg', '17.jpg', '10.jpg', '1.jpg', '7.jpg', '14.jpg', '6.jpg', '13.jpg', '16.jpg', '21.jpg', '5.jpg', '9.jpg', '22.jpg', '25.jpg', '24.jpg', '23.jpg', '19.jpg', '0.jpg', '12.jpg', '2.jpg', '4.jpg', '15.jpg', '8.jpg']
['880.jpg', '876.jpg', '883.jpg', '871.jpg', '874.jpg', '875.jpg', '877.jpg', '878.jpg', '890.jpg', '891.jpg', '889.jpg', '884.jpg', '879.jpg', '887.jpg', '873.jpg', '882.jpg', '881.jpg', '888.jpg', '872.jpg', '886.jpg', '885.jpg']
['2877.jpg', '2867.jpg', '2873.jpg', '2878.jpg', '2881.jpg', '2871.jpg', '2868.jpg', '2870.jpg', '2880.jpg', '2883.jpg', '2887.jpg', '2874.jpg', '2875.jpg', '2879.jpg', '2876.jpg', '2869.jpg', '2884.jpg', '2882.jpg', '2872.jpg', '2886.jpg', '2885.jpg']


In [79]:
## DISPLAYING THE NUMBER OF IMAGES FOR EACH CLASSES
print('Number of cats train image files:', len(tr_cats_fnames))
print('Number of dogs train image files:', len(tr_dogs_fnames))
print('Number of cats test image files:', len(ts_cats_fnames))
print('Number of dogs test image files:', len(ts_dogs_fnames))

Number of cats train image files: 26
Number of dogs train image files: 26
Number of cats test image files: 21
Number of dogs test image files: 21


In [80]:
## RESCALING AND PREPROCESSING OF THE IMAGES USING IMAGE DATA GENERATOR
train_datagen = ImageDataGenerator(rescale = 1/255)
test_datagen = ImageDataGenerator(rescale = 1/255)

In [81]:
## GENERATING THE TRAIN AND TEST IMAGES
train_data = train_datagen.flow_from_directory(train_dir, target_size = (150, 150), class_mode = 'binary')
test_data = test_datagen.flow_from_directory(test_dir, target_size = (150, 150), class_mode = 'binary')

Found 52 images belonging to 2 classes.
Found 42 images belonging to 2 classes.


In [82]:
## STORING THE DATA AND TARGET VALUES
X_train, y_train = next(train_data)

X_test, y_test = next(test_data)

In [83]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(32, 150, 150, 3)
(32,)
(32, 150, 150, 3)
(32,)


In [84]:
## DEFINING THE CLASS LABELS FOR EACH CLASSES
train_data.class_indices

{'train_cats': 0, 'train_dogs': 1}

In [85]:
## BUILDING THE CNN
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = (150, 150, 3)))
model.add(MaxPool2D(pool_size = 2))
model.add(Conv2D(64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = 2))
model.add(Conv2D(128, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = 2))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 150, 150, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 75, 75, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 75, 75, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 37, 37, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 37, 37, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 18, 18, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 41472)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │       2,654,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,747,585 (10.48 MB)

 Trainable params: 2,747,585 (10.48 MB)

 Non-trainable params: 0 (0.00 B)

In [86]:
## MODEL COMPILATION
model.compile(optimizer = 'RMSProp', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [89]:
## MODEL TRAINING
hist = model.fit(train_data, epochs = 15, validation_data = test_data)

Epoch 1/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.5417 - loss: 0.6193 - val_accuracy: 0.5714 - val_loss: 0.6859
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.8221 - loss: 0.6189 - val_accuracy: 0.5476 - val_loss: 0.6781
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - accuracy: 0.8256 - loss: 0.5166 - val_accuracy: 0.5000 - val_loss: 0.7990
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.6910 - loss: 0.5358 - val_accuracy: 0.5238 - val_loss: 0.6883
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.7708 - loss: 0.4494 - val_accuracy: 0.5238 - val_loss: 0.7656
Epoch 6/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.7372 - loss: 0.4696 - val_accuracy: 0.5238 - val_loss: 0.8505
Epoch 7/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - accuracy: 0.8103 - loss: 0.4415 - val_accuracy: 0.6429 - val_loss: 0.7000
Epoch 8/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9282 - loss: 0.2851 - val_accuracy: 0.4762 - val_loss: 1.1390
Epoch 9/15
2/2 ━

In [91]:
## PREDICTION FROM THE TRAIN_DATA

y_pr = np.round(model.predict(X_train))
y_pr = y_pr.reshape((32, ))
print('Predicted classes', y_pr)
print('Target classes', y_train)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step
Predicted classes [0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 1.]
Target classes [0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 1.]


In [92]:
## PREDICTION FROM THE TEST_DATA
y_pr = np.round(model.predict(X_test))
y_pr = y_pr.reshape((32, ))
print('Predicted classes', y_pr)
print('Target classes', y_test)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 616ms/step
Predicted classes [0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 0. 1. 0. 0. 1. 0.]
Target classes [1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 0. 1. 1. 0. 0. 1. 0. 1.]


In [96]:
## MODEL EVALUATION
accuracy = accuracy_score(y_pr, y_test)
accuracy

0.5

In [97]:
## PRINTING THE CONFUSION_MATRIX
conf_matrix = confusion_matrix(y_pr, y_test)
conf_matrix

array([[8, 8],
       [8, 8]])